In [1]:
import pickle
with open("../testing_summary.pickle", "rb") as fh:
    test_runs = pickle.load(fh)

In [2]:
from itertools import groupby
import pandas
import junitparser


rows = []
for name, runs in test_runs.items():
    env, _, run_i = name.rpartition("-")
    try:
        int(run_i)
    except ValueError:
        env = name

    for date, run in runs.items():
        for suite in run:
            for test in suite:
                if not test.classname:
                    continue

                if not test.result:
                    fail = 0
                elif any(
                    isinstance(result, (junitparser.Failure, junitparser.Error))
                    for result in test.result
                ):
                    fail = 1
                else:
                    # Skip or xfail
                    continue
                        
                rows.append({"test": f"{test.classname}::{test.name}", "env": env, "date": date, "fail": fail})

df = pandas.DataFrame(rows)
df

,test,env,date,fail
0,distributed.cli.tests.test_dask_scheduler::tes...,windows-latest-3.9,2021-10-02T11:05:01Z,0
1,distributed.cli.tests.test_dask_scheduler::tes...,windows-latest-3.9,2021-10-02T11:05:01Z,0
2,distributed.cli.tests.test_dask_scheduler::tes...,windows-latest-3.9,2021-10-02T11:05:01Z,0
3,distributed.cli.tests.test_dask_scheduler::tes...,windows-latest-3.9,2021-10-02T11:05:01Z,0
4,distributed.cli.tests.test_dask_scheduler::tes...,windows-latest-3.9,2021-10-02T11:05:01Z,0
...,...,...,...,...
762624,distributed.tests.test_worker_client::test_tim...,macos-latest-3.7,2021-09-28T16:11:17Z,0
762625,distributed.tests.test_worker_client::test_sec...,macos-latest-3.7,2021-09-28T16:11:17Z,0
762626,distributed.tests.test_worker_client::test_com...,macos-latest-3.7,2021-09-28T16:11:17Z,0
762627,distributed.tests.test_worker_client::test_wor...,macos-latest-3.7,2021-09-28T16:11:17Z,0


In [3]:
df.groupby("date").count().test

date
2021-09-28T16:11:17Z     10755
2021-09-28T17:33:42Z     10584
2021-09-28T18:38:44Z      7390
2021-09-29T08:01:25Z     10902
2021-09-29T08:36:27Z     10896
2021-09-29T09:12:42Z     10903
2021-09-29T09:22:11Z      4618
2021-09-29T10:26:40Z     10903
2021-09-29T10:54:07Z     10589
2021-09-29T13:24:40Z     10448
2021-09-29T14:25:09Z     10936
2021-09-29T18:04:25Z     10854
2021-09-30T11:27:16Z     10561
2021-09-30T11:54:13Z      5847
2021-09-30T12:03:10Z     10610
2021-09-30T12:54:48Z     10723
2021-09-30T13:11:25Z      3420
2021-09-30T13:26:55Z     10724
2021-09-30T13:31:51Z     10908
2021-09-30T14:21:55Z     10920
2021-09-30T14:55:03Z     10782
2021-09-30T16:18:52Z     10753
2021-09-30T16:52:14Z     10945
2021-09-30T17:23:00Z     11105
2021-09-30T18:01:31Z     10937
2021-09-30T18:39:48Z     10478
2021-09-30T20:55:22Z     10943
2021-09-30T21:04:12Z      4470
2021-09-30T21:56:30Z     10786
2021-09-30T22:17:30Z      2326
2021-09-30T22:27:45Z     10775
2021-09-30T23:07:34Z     11103
202

In [4]:
df2 = df[df.date == '2021-10-02T11:05:01Z']

In [9]:
fails = df2.groupby(["test", "env"]).sum(["fail"]).reset_index()
fails = fails.pivot(index="test", columns="env", values="fail").fillna(0).astype(int)
fails["TOTAL"] = fails.sum(axis=1)
fails = fails[fails.TOTAL > 0]
fails = fails.sort_values("TOTAL")[::-1]
fails.replace(0, "")

env,macos-latest-3.7,macos-latest-3.8,macos-latest-3.9,ubuntu-latest-3.7,ubuntu-latest-3.8,ubuntu-latest-3.9,windows-latest-3.7,windows-latest-3.8,windows-latest-3.9,TOTAL
test,,,,,,,,,,
distributed.tests.test_scheduler::test_gather_failing_cnn_error,20,20,20,19,20,20,20,20,,159
distributed.tests.test_stress::test_stress_creation_and_deletion,6,,,15,,,20,20,3,64
distributed.tests.test_worker::test_worker_reconnects_mid_compute_multiple_states_on_scheduler,,2,1,,,1,11,20,1,36
distributed.tests.test_worker::test_worker_reconnects_mid_compute,1,1,,,,,3,20,,25
distributed.tests.test_multi_locks::test_timeout_wake_waiter,,,,,,,,17,,17
distributed.tests.test_client::test_client_timeout,3,,2,,,,,,,5
distributed.diagnostics.tests.test_worker_plugin::test_WorkerPlugin_overwrite,,,1,2,1,,,,,4
distributed.tests.test_client::test_aliases_2,1,1,,,,,,,,2
distributed.cli.tests.test_dask_worker::test_nanny_worker_port_range_too_many_workers_raises,1,,,1,,,,,,2


In [6]:
totals = df2.groupby(["test", "env"]).count().fail.to_frame("total").reset_index()
totals = totals.pivot(index="test", columns="env", values="total")
totals["TOTAL"] = totals.sum(axis=1)
totals = totals.loc[fails.index]
((fails / totals).round(3) * 100).replace(0, "").fillna("skip/xfail")

env,macos-latest-3.7,macos-latest-3.8,macos-latest-3.9,ubuntu-latest-3.7,ubuntu-latest-3.8,ubuntu-latest-3.9,windows-latest-3.7,windows-latest-3.8,windows-latest-3.9,TOTAL
test,,,,,,,,,,
distributed.tests.test_scheduler::test_gather_failing_cnn_error,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,,89.3
distributed.tests.test_stress::test_stress_creation_and_deletion,30.0,,,78.9,,,100.0,100.0,15.8,36.0
distributed.tests.test_worker::test_worker_reconnects_mid_compute_multiple_states_on_scheduler,,10.0,5.0,,,5.0,55.0,100.0,5.3,20.2
distributed.tests.test_worker::test_worker_reconnects_mid_compute,5.0,5.0,,,,,15.0,100.0,,14.0
distributed.tests.test_multi_locks::test_timeout_wake_waiter,,,,,,,,85.0,,9.6
distributed.tests.test_client::test_client_timeout,15.0,,10.0,,,,,,,2.8
distributed.diagnostics.tests.test_worker_plugin::test_WorkerPlugin_overwrite,,,5.0,10.5,5.0,,,,,2.2
distributed.tests.test_client::test_aliases_2,5.0,5.0,,,,,,,,1.1
distributed.cli.tests.test_dask_worker::test_nanny_worker_port_range_too_many_workers_raises,5.0,,,5.3,,,,,,1.1


In [10]:
totals.fillna(0).astype(int)

env,macos-latest-3.7,macos-latest-3.8,macos-latest-3.9,ubuntu-latest-3.7,ubuntu-latest-3.8,ubuntu-latest-3.9,windows-latest-3.7,windows-latest-3.8,windows-latest-3.9,TOTAL
test,,,,,,,,,,
distributed.tests.test_scheduler::test_gather_failing_cnn_error,20,20,20,19,20,20,20,20,19,178
distributed.tests.test_stress::test_stress_creation_and_deletion,20,20,20,19,20,20,20,20,19,178
distributed.tests.test_worker::test_worker_reconnects_mid_compute_multiple_states_on_scheduler,20,20,20,19,20,20,20,20,19,178
distributed.tests.test_worker::test_worker_reconnects_mid_compute,20,20,20,19,20,20,20,20,19,178
distributed.tests.test_multi_locks::test_timeout_wake_waiter,20,20,20,19,20,20,20,20,19,178
distributed.tests.test_client::test_client_timeout,20,20,20,19,20,20,20,20,19,178
distributed.diagnostics.tests.test_worker_plugin::test_WorkerPlugin_overwrite,20,20,20,19,20,20,20,20,19,178
distributed.tests.test_client::test_aliases_2,20,20,20,19,20,20,20,20,19,178
distributed.cli.tests.test_dask_worker::test_nanny_worker_port_range_too_many_workers_raises,20,20,20,19,20,20,20,20,19,178
